In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.tgz
!pip install -q findspark

# # Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [993 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-20 19:31:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2023-04-20 19:31:06 (10.6 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...|2015-08-31 00:00:00|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...| 

### Create DataFrames to match tables

In [5]:
# Read in the Review dataset as a DataFrame


In [6]:
# Fixing date
from pyspark.sql.functions import to_date

# Fix the date 
df.select(to_date(df["review_date"], "yyyy-MM-dd")).show()

+--------------------------------+
|to_date(review_date, yyyy-MM-dd)|
+--------------------------------+
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
|                      2015-08-31|
+--------------------------------+
only showing top 20 rows



In [7]:
 # Keeping new column with date
df = df.withColumn("date", to_date(df["review_date"], "yyyy-MM-dd"))
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|      date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...|2015-08-31 00:00:00|2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001

In [8]:
# Create the customers_table DataFrame

customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")

customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45978717|             2|
|   43622307|             1|
|     740678|             4|
|   29045703|             1|
|   52484548|             3|
|   42560427|             4|
|   17067926|             1|
|   10093406|             1|
|   44979559|             1|
|   19432125|             1|
|   26079415|             1|
|   29931671|             1|
|   12945150|             3|
|   12036434|             1|
|   14230926|             1|
|   45015535|             1|
|   39064792|             1|
|   20709090|             1|
|   45074906|             1|
|   26767269|             2|
+-----------+--------------+
only showing top 20 rows



In [9]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select('product_id','product_title').drop_duplicates()
products_df.show(5)


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000JNNWQ2|IRWIN Tools VISE-...|
|B000OECQOK|Titan 17005 Stain...|
|B0045W0MD4|The American Red ...|
|B00DQQ9ET8|de.power DP-010AA...|
|B001P30BQE|DEWALT DG5204 Pro...|
+----------+--------------------+
only showing top 5 rows



Problem with product_id B00AT8BXDG having two observations.
 

In [10]:
product_check = products_df.groupby("product_id").count()

df2 = product_check.filter("count>1").select(['product_id', 'count'])
df2.show(n=df2.count(), truncate=False)

+----------+-----+
|product_id|count|
+----------+-----+
|B00AT8BXDG|2    |
+----------+-----+



In [11]:
prob= products_df.filter("product_id=='B00AT8BXDG'")
prob.show(truncate=False)

+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                                                                                                                            |
+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|B00AT8BXDG|1600Lumen CREE XML T6 LED Zoomable Adjustable Focus 3 Modes Headlamp Head Torch Light+ Charger(100V-240V)\tTools\t5\t0\t0\tN\tY\tWorked great\tIt worke

There are two product elements under one product_id the first contains part of the review.  I'm resetting both to a new product title. 

In [12]:

old1 = '"1600Lumen CREE XML T6 LED Zoomable Adjustable Focus 3 Modes Headlamp Head Torch Light+ Charger(100V-240V)\tTools\t5\t0\t0\tN\tY\t"1600Lumen CREE XML T6 LED Zoomable Adjustable Focus 3 Modes Headlamp'
new = "1600Lumen CREE XML T6 LED Zoomable Adjustable Focus 3 Modes Headlamp Head Torch Light+ Charger(100V-240V)"
old2 = '1600Lumen CREE XML T6 LED Zoomable Adjustable Focus 3 Modes Headlamp Head Torch Light+ Charger(100V-240V)\tTools\t5\t0\t0\tN\tY\tWorked great\tIt worked great until I lost it somewhere does need special batteries. I am going to order me a new one later.\t2013-12-15'

products_df = products_df.replace(old1,new)
products_df = products_df.replace(old2,new)

products_df.filter("product_id=='B00AT8BXDG'").show(truncate=False)


+----------+---------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                            |
+----------+---------------------------------------------------------------------------------------------------------+
|B00AT8BXDG|1600Lumen CREE XML T6 LED Zoomable Adjustable Focus 3 Modes Headlamp Head Torch Light+ Charger(100V-240V)|
|B00AT8BXDG|1600Lumen CREE XML T6 LED Zoomable Adjustable Focus 3 Modes Headlamp Head Torch Light+ Charger(100V-240V)|
+----------+---------------------------------------------------------------------------------------------------------+



In [13]:
products_df = products_df.select('product_id','product_title').drop_duplicates()

In [14]:
# Create the review_id_table DataFrame. 

review_id_df = df.select('review_id','customer_id','product_id','product_parent','date').withColumnRenamed("date", "review_date")

review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2UM5QMHBHC90Q|   15785389|B00H5U9ZD6|     115362950| 2015-08-31|
|  RF0D1LEIF6L7|   47910848|B001TJGCS0|     570955425| 2015-08-31|
|  RM6YKIWQVNSY|   36328996|B000NIK8JW|     128843593| 2015-08-31|
|R1RL3L68ASPS36|   51785809|B008ZYRGUA|     407828107| 2015-08-31|
|R1U4XFBFAG34CY|   40757491|B00K5CA0GC|     490746675| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [15]:
# Create the vine_table. DataFrame
vine_df = df.select('review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase')
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

In [16]:
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [17]:
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import col

vine_df=vine_df.withColumn("star_rating", col("star_rating").cast("int"))

vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [18]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://cwcmod17chanllenge.c4ay8x7sqcwl.us-east-2.rds.amazonaws.com:5432/cwcmod17chanllenge"
config = {"user":"root", 
          "password": "---", 
          "driver":"org.postgresql.Driver"}

In [20]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [21]:
# Write products_df to table in RDS
# about 3 min

products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [22]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [23]:
# Write vine_df to table in RDS
# 11 minutes
# fixing data type issue


vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)